# Resumo do código Alteração

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [11]:
%%time
import pandas as pd
import numpy as np
import datetime
import clickhouse_connect

CPU times: total: 0 ns
Wall time: 0 ns


---

#  <span style="color:Blue">Juntar Ficheiros</span>

- Importar do Clickhouse

In [4]:
import clickhouse_connect
client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX',
                                       database = "Delta")

In [7]:
%%time
#Ler o ficheiro

dfClick = client.query_df('SELECT * FROM Delta_dbd_120dias')

dfClick = dfClick.sort_values(by = "DATA")
dfClick = dfClick[dfClick.DATA != dfClick.DATA.unique()[0]]

CPU times: total: 34.8 s
Wall time: 1min 22s


In [8]:
dfClick.DATA.unique()

<DatetimeArray>
['2023-07-02 00:00:00', '2023-07-03 00:00:00', '2023-07-04 00:00:00',
 '2023-07-05 00:00:00', '2023-07-06 00:00:00', '2023-07-07 00:00:00',
 '2023-07-08 00:00:00', '2023-07-09 00:00:00', '2023-07-10 00:00:00',
 '2023-07-11 00:00:00',
 ...
 '2023-10-21 00:00:00', '2023-10-22 00:00:00', '2023-10-23 00:00:00',
 '2023-10-24 00:00:00', '2023-10-25 00:00:00', '2023-10-26 00:00:00',
 '2023-10-27 00:00:00', '2023-10-28 00:00:00', '2023-10-29 00:00:00',
 '2023-10-30 00:00:00']
Length: 120, dtype: datetime64[s]

- Importar ficheiros novos

### Excel

In [16]:
# dfNovo = pd.read_excel("D:\\B&N Dados\\Delta\\Diário\\31-10.xlsb") #xlsx
dfNovo = pd.read_excel("C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Diario\\31-10.xlsb")

dfNovo = dfNovo[['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                 "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)", "FLUXO"]]

# Renomear colunas e criar as do dia

dfNovo = dfNovo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", 
                                "SOH": "STOCK_1_Dias_Antes",
                                "INTRANSIT": "INTRANSIT_1_Dias_Antes", 
                                "EXPECTED": "EXPECTED_1_Dias_Antes", 
                                "PRES_STOCK": "PRES_STOCK_1_Dias_Antes"})

dfNovo["SELLOUT_1_Dias_Antes"] = np.where(dfNovo["SELLOUT_1_Dias_Antes"]<0, 0, dfNovo["SELLOUT_1_Dias_Antes"])

#DATETIME
# se xlsx
#dfNovo['DATA'] = pd.to_datetime(dfNovo['DATA'], format='%d-%m-%Y') 

# se xlsb
dfNovo['DATA'] = pd.to_datetime(dfNovo['DATA'], unit='D', origin=datetime.datetime(1899, 12, 30)) 

### Clickhouse

- Produtos específicos

- Juntar

In [41]:
dfDiario = pd.concat([dfClick, dfNovo], ignore_index=True, join='outer')

dfFinal = dfDiario.copy()
dfFinal['DATA'] = pd.to_datetime(dfFinal['DATA'], format='%Y-%m-%d') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3604106 entries, 0 to 3604105
Columns: 146 entries, DATA to Media_Sellout_Mensal_Mes_Anterior
dtypes: float64(126), object(4), string(16)
memory usage: 3.9+ GB


- Vendedores

In [49]:
#dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor=pd.read_excel("C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")

dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

dfFinal = dfFinal.drop(columns="Vendedor")

# Criar coluna de reposição
dfFinal = pd.merge(dfFinal, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

In [50]:
dfFinal['SELLOUT'] = dfFinal.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfFinal['STOCK'] = dfFinal.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)
dfFinal['INTRANSIT'] = dfFinal.groupby(["STORE","EAN"])['INTRANSIT_1_Dias_Antes'].shift(-1)
dfFinal['EXPECTED'] = dfFinal.groupby(["STORE","EAN"])['EXPECTED_1_Dias_Antes'].shift(-1)
dfFinal['PRES_STOCK'] = dfFinal.groupby(["STORE","EAN"])['PRES_STOCK_1_Dias_Antes'].shift(-1)

# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA
> - PRÉ_ROTURA

In [51]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)
dfFinal["ROTURA_1_Dias_Antes"] = np.where((dfFinal["STOCK_1_Dias_Antes"] <= 0) & (dfFinal["PRES_STOCK_1_Dias_Antes"] > 0), 1, 0)


# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)
dfFinal["PRE_ROTURA_1_Dias_Antes"] = (dfFinal["STOCK_1_Dias_Antes"] < dfFinal["PRES_STOCK_1_Dias_Antes"]).astype(int)

# Métricas 1, 4, 5 e 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [52]:
# Quantos dias antes:

diasMet = [4, 5, 10]

# Função para colunas de dias anteriores

def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,f'{coluna}_{a}_Dias_Antes'] = valores

---

> - STOCK
> - SELLOUT
> - INTRANSIT
> - EXPECTED 
> - STK
> - FORNECIMENTO
> - CICLOS
> - Adequação

In [53]:
%%time
dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]
dfFinal["STK_1_Dias_Antes"] = dfFinal["STOCK_1_Dias_Antes"] + dfFinal["INTRANSIT_1_Dias_Antes"] + dfFinal["EXPECTED_1_Dias_Antes"]

dfFinal["FORNECIMENTO"] = dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]
dfFinal["FORNECIMENTO_1_Dias_Antes"] = dfFinal["INTRANSIT_1_Dias_Antes"] + dfFinal["EXPECTED_1_Dias_Antes"]

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]
dfFinal["CICLOS_1_Dias_Antes"] = dfFinal["STOCK_1_Dias_Antes"]/dfFinal["PRES_STOCK_1_Dias_Antes"]
dfFinal["CICLOS"] = dfFinal["CICLOS"].fillna(0)
dfFinal.loc[np.isinf(dfFinal['CICLOS']), "CICLOS"] = 0

dfFinal["Adequacao"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))
dfFinal["Adequacao_1_Dias_Antes"]= np.where(dfFinal["CICLOS_1_Dias_Antes"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS_1_Dias_Antes"] <= 1.1) & (dfFinal["INTRANSIT_1_Dias_Antes"]+dfFinal["EXPECTED_1_Dias_Antes"]+dfFinal["STOCK_1_Dias_Antes"]>=dfFinal["PRES_STOCK_1_Dias_Antes"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS_1_Dias_Antes"] <= 1.1) & (dfFinal["INTRANSIT_1_Dias_Antes"]+dfFinal["EXPECTED_1_Dias_Antes"]+dfFinal["STOCK_1_Dias_Antes"]<dfFinal["PRES_STOCK_1_Dias_Antes"]), "Stock Insuf c Forn Desadequado", 
                      "")))


for i in diasMet:   
    dias(dfFinal, i, "STOCK")
    dias(dfFinal, i, "SELLOUT")
    dias(dfFinal, i, "INTRANSIT")
    dias(dfFinal, i, "EXPECTED")
    dias(dfFinal, i, "STK")
    dias(dfFinal, i, "FORNECIMENTO")
    dias(dfFinal, i, "CICLOS")
    dias(dfFinal, i, "Adequacao")

CPU times: total: 24.3 s
Wall time: 1min 11s


> - MSA
> - Balance: sellout / soma stock disponível mais transito.

In [54]:
%%time
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=10).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")
    

dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=20).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in diasMet:
    
    
    valores = i * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores

    

dfFinal.loc[dfFinal.STK == 0, "Balance"] = 2
dfFinal.Balance = dfFinal.Balance.fillna(2)

CPU times: total: 11.7 s
Wall time: 34 s


# Colunas de métricas 30, 60 e 120 dias antes

In [55]:
diasMetHist = [30, 60, 120]

### Percentagens

>- Balance raw

In [56]:
%%time
dfFinal["Balance_Raw"] =  dfFinal["SELLOUT"] / dfFinal["STK"]
dfFinal.loc[dfFinal.STK == 0, "Balance_Raw"] = 2

dfFinal["Balance_Raw_Count1"] = np.where(dfFinal["Balance_Raw"] < 0.5, 1, 0)

dfFinal["Balance_Raw_Count2"] = np.where(dfFinal["Balance_Raw"] < 0.8, 1, 0)

for i in diasMetHist:
    dfFinal[f"Percentagem_Balance_Raw_Count1_{i}"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Balance_Raw_Count1'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())
    dfFinal[f"Percentagem_Balance_Raw_Count2_{i}"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Balance_Raw_Count2'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())

CPU times: total: 7.94 s
Wall time: 24.2 s


> - Efeito fim de semana

In [57]:
%%time
#dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
#dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()
dfFinal['SELLOUT_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["SELLOUT_1_Dias_Antes"].copy()
dfFinal['SELLOUT_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["SELLOUT_1_Dias_Antes"].copy()

dfFinal['SELLOUT_fds_Medio'] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())
dfFinal['SELLOUT_semana_Medio'] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())


for i in diasMetHist:
    dfFinal[f"Percentagem_Efeito_Fds_{i}"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())/
                                              (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())))-1)


CPU times: total: 10.6 s
Wall time: 36.1 s


> - Volatilidade
> - Rotura 
> - Supply
> - Percentagem de dias em Stock Borderline
> - Percentagem de dias de Linear Incompleto
> - Percentagem de dias sem vendas
> - Tempo indisponível
> - Vendas perdidas

In [58]:
%%time
dfFinal=dfFinal.copy()
# Sempre que é pedido abastecimento, fazer com que seja 1
dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED_1_Dias_Antes"]==0) & (dfFinal["EXPECTED"]>0), 1, 0)
dfFinal["Percentagem_Stock_Borderline_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<0.2*dfFinal["PRES_STOCK_1_Dias_Antes"], 1, 0)
dfFinal["Percentagem_Linear_Incompleto_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<dfFinal["PRES_STOCK_1_Dias_Antes"], 1, 0)
dfFinal["Sem_Vendas_1_Dias_Antes"] = np.where(dfFinal["SELLOUT_1_Dias_Antes"] == 0, 1, 0)
dfFinal["Dias_Indisponivel_1_Dias_Antes"] = np.where(dfFinal["ROTURA_1_Dias_Antes"]==1, 1, 0)

dfFinal['ROTURA_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["ROTURA_1_Dias_Antes"].copy()
dfFinal['ROTURA_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["ROTURA_1_Dias_Antes"].copy()

for i in diasMetHist:
    dfFinal[f"Percentagem_Volatilidade_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes']\
                                                .transform(lambda x: x.rolling(window=i, min_periods=1).std()) /
                                                dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes']\
                                                .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes']\
                                           .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes']\
                                           .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Supply_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply']\
                                          .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Percentagem_Dias_Stock_Borderline_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Stock_Borderline_1_Dias_Antes']\
                                                         .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Percentagem_Dias_Linear_Incompleto_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Linear_Incompleto_1_Dias_Antes']\
                                                          .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Dias_Sem_Vendas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas_1_Dias_Antes']\
                                                   .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Dias_Indisponivel_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Dias_Indisponivel_1_Dias_Antes']\
                                                     .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Vendas_Perdidas_em_{i}_Dias"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum())\
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())) \
                                                  + 
                                               (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum()) \
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())))
    
dfFinal.Percentagem_Volatilidade_60 = dfFinal.Percentagem_Volatilidade_60.fillna(0)

CPU times: total: 1min 3s
Wall time: 2min 43s


> - Tempo médio inter-supply

In [59]:
%%time
dfFinal=dfFinal.copy()
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

for i in diasMetHist:
    dfFinal[f"Percentagem_InterSupplyMed_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))


CPU times: total: 9.45 s
Wall time: 25.1 s


---

 ### <font color=green> Função Contagem

In [60]:
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def contagem5dias(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})


# Define a function to calculate weekday and weekend counts
def contagem3dias(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(3):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA3": weekday_count, "CONTAGEM_FIMSEMANA3": weekend_count})

> - Smart 3 dias

In [61]:
%%time

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA3", "CONTAGEM_FIMSEMANA3"]] = dfFinal["DATA"].apply(contagem3dias)
dfFinal['CONTAGEM_FIMSEMANA3'] = dfFinal['CONTAGEM_FIMSEMANA3'].shift(-1)
dfFinal['CONTAGEM_SEMANA3'] = dfFinal['CONTAGEM_SEMANA3'].shift(-1)

dfFinal["Balance_Smart3"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA3'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA3']))
     / dfFinal["STK"])

dfFinal["Balance_Smart3"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart3"])
dfFinal["Balance_Smart_3_Dias_Antes"] = dfFinal["Balance_Smart3"].shift(2)


CPU times: total: 1min 38s
Wall time: 5min 21s


> - Smart 5 dias antes

In [62]:
%%time

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(contagem5dias)
dfFinal['CONTAGEM_FIMSEMANA'] = dfFinal['CONTAGEM_FIMSEMANA'].shift(-1)
dfFinal['CONTAGEM_SEMANA'] = dfFinal['CONTAGEM_SEMANA'].shift(-1)


dfFinal["Balance_Smart"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK"])

dfFinal["Balance_Smart"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart"])
dfFinal["Balance_Smart_5_Dias_Antes"] = dfFinal["Balance_Smart"].shift(4)


CPU times: total: 2min 3s
Wall time: 5min 32s


> - Dias para rotura de Stock

In [63]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STK_1_Dias_Antes"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_para_Rotura_Stock_5_Dias_Antes"] = dfFinal["Dias_para_Rotura_Stock"].shift(4)

> - Dias para rotura de Linear

In [64]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duracao_Linear'] = dfFinal["PRES_STOCK"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_Duracao_Linear_5_Dias_Antes"] = dfFinal["Dias_Duracao_Linear"].shift(4)

### Texto

> - Desempenho
> - Volatilidade de Sellout
> - Sensibilidade
> - PS Classificação
> - Risco de Rotura

In [65]:
%%time
dfFinal["Desempenho_de_Vendas_60"] = np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>=0) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.151), "Vendas com desempenho satisfatório",
                                     np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.151) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.317), "Vendas com desempenho mediano",
                                     np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.317) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.433), "Vendas com desempenho insatisfatório",
                                              "Vendas com desempenho muito insatisfatório")))


                                     #np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.433) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=1), 

dfFinal["Volatilidade_SELLOUT"] = np.where(dfFinal["Percentagem_Volatilidade_60"] > 1.095, " e com alta volatilidade de sellout.",
                                  np.where(dfFinal["Percentagem_Volatilidade_60"] > 0.91 , " e com média volatilidade de sellout.",
                                  np.where(dfFinal["Percentagem_Volatilidade_60"] > 0.76 , " e com baixa volatilidade de sellout.",
                                                                                           " e com muito baixa volatilidade de sellout.")))


dfFinal["Sensibilidade_Rotura"] = np.where(dfFinal["Percentagem_Roturas_60"]>0.33, " Produto com muito elevada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.166666, " Produto com elevada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.017, " Produto com moderada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0, " Produto com baixa propensão para rotura",
                                 " Produto com muito baixa ou nula propensão para rotura"))))

dfFinal["Order_Sensibilidade_Rotura"] = np.where(dfFinal["Percentagem_Roturas_60"]>0.33, 5,
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.166666, 4,
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.017, 3,
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0, 2,
                                 1))))

dfFinal["PS_Classificacao_60"] = np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.51) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==1) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS excessivo.",
                                 np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.51) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==0) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS excessivo.",
                                 np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.31) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==1) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS moderadamente excessivo.",
                                 np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] < 0.07) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]<0.8) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<0.07), " e com PS escasso.", 
                                                                                                                                                                                                            " e com PS equilibrado."))))

dfFinal["RISCO"] = np.where((dfFinal.Balance_Smart > 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.2) & (dfFinal.Percentagem_Roturas_120 > 5), 1,
                   np.where((dfFinal.Balance_Smart > 1) & (dfFinal.Balance_Smart < 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.6) & (dfFinal.Percentagem_Roturas_120 < 3), 2, 3))

dfFinal["RISCO_ROTURA"] = np.where((dfFinal.Balance_Smart >= 12.5) | (dfFinal.Balance_Smart < 0), " O produto apresenta hoje muito elevada probabilidade de rotura a 5 dias caso não haja fornecimento.", 
                          np.where((dfFinal.Balance_Smart >= 1.1) & (dfFinal.Balance_Smart < 12.5), " O produto apresenta hoje elevada probabilidade de rotura a 5 dias caso não haja fornecimento.",
                          np.where((dfFinal.Balance_Smart >= 0.833) & (dfFinal.Balance_Smart < 1.1), " O produto apresenta hoje média probabilidade de rotura a 5 dias caso não haja fornecimento.",
                          np.where((dfFinal.Balance_Smart >= 0.087) & (dfFinal.Balance_Smart < 0.833), " O produto apresenta hoje baixa probabilidade de rotura a 5 dias caso não haja fornecimento.", 
                                                                                                         " O produto apresenta hoje muito baixa ou nula probabilidade de rotura a 5 dias caso não haja fornecimento."))))         

CPU times: total: 1.5 s
Wall time: 5.84 s


> - Conclusão

In [66]:
dfFinal["Conclusao1"] = dfFinal["Desempenho_de_Vendas_60"] + dfFinal["Volatilidade_SELLOUT"] 
dfFinal["Conclusao2"] = dfFinal["Sensibilidade_Rotura"] + dfFinal["PS_Classificacao_60"] 
dfFinal["Conclusao3"] = dfFinal["RISCO_ROTURA"]

# <font color=blue>Escrever

- Dias certos

In [67]:
dfFinal["ROTURA"] = np.where(dfFinal["ROTURA"]==1, "ROTURA", "PRESENTE")
dfFinal['Subinsignia'] = dfFinal['STORE_NAME'].str.split().str[0]
dfFinalEscrever = dfFinal.copy()

dfEscrever120 = dfFinalEscrever[dfFinalEscrever.DATA.between(dfFinalEscrever.DATA.unique()[-121],dfFinalEscrever.DATA.unique()[-1])].copy()
dfFinalEscrever = dfFinalEscrever.dropna(subset=['SELLOUT'])

# Ficheiro Dia, Mês e 120 dias
dfEscreverDia = dfFinalEscrever[dfFinalEscrever.DATA == dfFinalEscrever.DATA.unique()[-1]].copy()
dfEscreverMes = dfFinalEscrever[dfFinalEscrever.DATA.between(dfFinalEscrever.DATA.unique()[-31],dfFinalEscrever.DATA.unique()[-1])].copy()

# Clickhouse

In [68]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX')


# Estabelecer a base a ser lida
baseCH = dfEscrever120.copy()

# Estabelecer a tabela a ser criada
tabela = "Delta_120dias"

In [69]:
%%time
#Estabelecer a base a ser lida

bases = [1, 2, 3]

for qualbase in bases:

    if qualbase == 1:
        baseCH = dfEscreverDia.copy()
        tabela = "Delta_dbd_Dia" # BaseGoChillDia

    elif qualbase == 2:
        baseCH = dfEscreverMes.copy()
        tabela = "Delta_dbd_Mes" # BaseGoChillMes

    elif qualbase == 3:
        baseCH = dfEscrever120.copy()
        tabela = "Delta_dbd_120dias" # BaseDelta120

    elif qualbase == 4:
        baseCH = dfNinjas.copy()
        tabela = "Delta_dbd_Audit" # BaseDeltaNinja

    ## Algumas alterações
    baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar a Data para datetime

    baseCH.columns = [unidecode(col) for col in baseCH.columns]        # Tirar acentos e afins dos nomes das colunas porque 
                                                                       # o Clickhouse não gosta


    ## Listas para definir os tipos de dados de cada coluna a inserit
    data = ["DATA"]
    texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
    inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
    floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


    ## Mudar inteiros para floats porque senão não pode haver missing values
    for col_name in inteiros:
        baseCH[col_name] = baseCH[col_name].astype(float)

    ## Missing values em strings também estragam tudo
    baseCH[texto] = baseCH[texto].fillna("-")
    baseCH[texto] = baseCH[texto].astype(str)

    #Função que vai fazer a schema
    def schema(lista, tipo):
        result_list = [f"{element} {tipo}" for element in lista]
        return result_list

    # Schema a ser feito
    data1 = schema(data, "Date")
    texto1 = schema(texto, "String")
    inteiros1 = schema(inteiros, "Float64")
    floats1 = schema(floats, "Float64")
    total = tuple(data1 + texto1 + inteiros1 + floats1)
    schema = ', '.join([column.replace("'", "") for column in total])

    # Split the input string by commas
    parts = schema.split(', ')
    # Process each part and wrap the first word in double quotes
    output_parts = []
    for part in parts:
        words = part.split()
        if words:
            first_word = words[0]
            remaining_words = ' '.join(words[1:])
            output_part = f'"{first_word}" {remaining_words}'
            output_parts.append(output_part)
    # Join the modified parts back into a string
    schema = ', '.join(output_parts)

    # Eliminar tabela que possa existir no CH com o mesmo nome
    client.command(f'DROP TABLE IF EXISTS {tabela}')

    # Criar tabela no CH
    client.command(f'''
        CREATE TABLE IF NOT EXISTS {tabela} (
            {schema}
            ) ENGINE = MergeTree
            ORDER BY (DATA)
    ''')


    # Exportar os dados para o clickhouse
    client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())
    print(f"A base {tabela} foi exportada para o clickhouse")

ProgrammingError: Unrecognized column 'Adequacao_4_Dias_Antes' in table Delta_dbd_Dia

- Passar para csv